# Bayesian Parameter Estimation on MBL Data

## July 7, 2016

In [4]:
import numpy as np
import pandas as pd
import emcee
import scipy.special
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
import seaborn as sns
rc = {'lines.linewidth': 1.5,
      'axes.labelsize' : 14,
      'axes.titlesize' : 18,
      'axes.facecolor' : 'EBEBEB',
      'axes.edgecolor' : '000000',
      'axes.linewidth' : 0.75,
      'axes.frameon' : True,
      'xtick.labelsize' : 11,
      'ytick.labelsize' : 11,
      'font.family' : 'Droid Sans',
      'grid.linestyle' : ':',
      'grid.color' : 'a6a6a6',
      'mathtext.fontset' : 'stixsans',
      'mathtext.sf' : 'sans'}
plt.rc('text.latex', preamble=r'\usepackage{sfmath}')
plt.rc('mathtext', fontset='stixsans', sf='sans') 
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)
sns.set_palette("deep", color_codes=True)

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#------------------------------------------------------------------------------- 
%matplotlib notebook

/Users/gchure/anaconda/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Our data consists of a set of fold-change in gene expression values from a set of experiments conducted at Woods Hole, MA at MBL Physiology. The purpose of this notebook is to perform a two-parameter fit on the active and inactive dissociation constants of the Lac repressor to DNA. Using Bayes' rule, we can say that

$$
\begin{align}
P(\{a\} \vert D, I) &\propto P(D \vert \{a\}, I) P(\{a\} \vert I) \\
& \propto P(D \vert K_a, K_i, I) P(K_a \vert I) P(K_i \vert I)
\end{align}
$$

where $\{a\}$ is the set of dissociation constants for the active and inactive repressors ($K_a$ and $K_i$ repsectively), $D$ are our data, and $I$ is the prior information we have with regards to the system. Note that in this case we are considering that the two dissociation constants are independent parameters. This is probably not correct as it would make logical sense that the ability to bind inducer in the active vs inactive state uses a similar binding pocket and are therefore related. For now, it is not obvious how these two parameters are related mathematically, so we will treat them as completely independent. We can be more explicity with the functional form of our posterior as follows: 

$$
P(K_a, K_i \vert D, I) \propto \prod\limits_{i=1}^N \exp\left\{\frac{fc_i - fc(R, \Delta\epsilon_r, K_a, K_i, \epsilon_{ai}))^2}{\sigma_i}\right\}\frac{1}{K_a^{max},
$$